# Python中的定点小数(`Decimal`)和分数(`Fraction`)对象, 以及支持的运算. 

In [1]:
import math, decimal, fractions; 
from decimal import Decimal; from fractions import Fraction; 

## 定点小数的构造, 及其与浮点小数的精度差异

### 浮点数的局限性
若采用浮点数表示一个有理数, 其最简分数的分母质因数包括$2_{(10)}$以外的质数时, 由于被表示的数在二进制下是无限循环小数, 因此采用浮点数保留其在二进制下的53位有效数字, 将截断后续的部分, 从而产生舍入误差. 

例如: 十进制下的两个有限小数$0.25_{(10)}$和$0.2_{(10)}$, 在二进制下将分别表示为

|十进制小数|十进制分数|二进制小数|
|:-|:-:|:-|
|$0.25_{(10)}$ (有限小数)|$1\over4$|$0.01_{2}$ (有限小数)|
|$0.2_{(10)}$ (有限小数)|$1\over5$|$0.\dot{0}01\dot{1}_{(2)}$ (无限循环小数)|

In [2]:
xFlt = 0.25; xFlt2Dec = Decimal(xFlt); xStr2Dec = Decimal("0.25"); 
print([x.as_integer_ratio() for x in [xFlt, xFlt2Dec, xStr2Dec]]); 
print(xFlt == xFlt2Dec, xFlt2Dec == xStr2Dec)

[(1, 4), (1, 4), (1, 4)]
True True


### 基于不同方法的`Decimal`定义及其机制
上述二进制下的无限循环小数, 分别用不同的方法定义, 定义机制各有差异: 
* 如果声明为`float`对象, 其在Python中的实际值与字面值将存在微小差异. 
> 例如, `yFlt`是通过直接在代码中输入十进制小数
> ```Python
> 0.2 #小数的两端没有任何形式的括注
> ```
> 定义得到的浮点数, 实际的表示结果是
> ```Python
> 0.200000000000000011102230246251565404236316680908203125
> ```
> 同时, 浮点数与浮点数计算, 过程中不仅会将这一误差放大, 还会引入新的误差并累计. 

* 如果用浮点数直接声明`Decimal`对象, 则会将浮点数按照**实际的表示结果**进行转换, 实际值与转换前的`float`一致, 仍与字面值存在微小差异. 
> 例如, `yFlt2Dec`是将(被解释器视为浮点数的)十进制小数直接用`Decimal`函数转换, 所构造的定点数对象. 
> ```Python
> Decimal(0.2) #把小数直接用Decimal(∙)括注
> ```
> 实际的表示结果也是
> ```Python
> 0.200000000000000011102230246251565404236316680908203125
> ```
> 此时浮点数被转化为十进制定点数, 与其他定点数的计算过程中, 原有的误差会被放大, 但不会引入新的误差. 

* 当它写作**被字符串引号括注后的**十进制小数的形式, 并被声明为`Decimal`对象, 则其在Python中的实际值将与字面值一致. 
> 例如, `yStr2Dec`是将十进制小数写作**被双引号括注后的字符串形式**后, 进一步, 通过
> ```Python
> Decimal("0.2") #先把小数用双引号括注, 再用Decimal(∙)括注
> ```
> 被`Decimal`函数声明, 所构造的定点数对象. 




In [3]:
yFlt = 0.2; yFlt2Dec = Decimal(0.2); yStr2Dec = Decimal("0.2"); 
print([y.as_integer_ratio() for y in [yFlt, yFlt2Dec, yStr2Dec]]); 

[(3602879701896397, 18014398509481984), (3602879701896397, 18014398509481984), (1, 5)]


### 基于不同定义方法所得`Decimal`对象的舍入误差特征
由上一计算单元的输出结果可见: 
* `yFlt`(即`0.2`) 和`yFlt2Dec`(即`Decimal(0.2)`)是相等的, 但都不等于数学意义上的0.2; 其中
    * `yFlt`是`float`对象, 在构造时即产生了舍入误差, 在后续计算中逐级传递, 并**与计算中新引入的误差结合**; 
    * `yFlt2Dec`(即`Decimal(0.2`)是**由`float`转换所得的**`Decimal`对象, 在构造时, 保留了浮点数`0.2`存在的舍入误差, 并且在将浮点转化为定点数的过程中, 将这个误差忠实地记录下来; \
        转化为定点数之后, 后续计算中不会引入新的误差, 只有**转换前的误差会被传递**; 
* 只有`yStr2Dec`(即`Decimal("0.2"`)是**由`str`构造所得的**`Decimal`对象, 其结果等于数学意义上的0.2, 在四则运算过程中产生或传递舍入误差的情形, 较`float`更少. 这是因为: 
> `Decimal`对象以十进制形式存储小数, 且对象在内存中的大小可以变化, 因此它可以表示不同精度的小数. 

由于以上特性, `Decimal`可被应用于金融计算领域. 以减少在频繁将不同数量级的数额进行累加, 或将相近的数额相减以计算差额的过程中, 采用`float`计算所引入和放大的舍入误差. 
> 在上述计算中, 若采用浮点数, 会发生"大数目'吃'小数目"现象. 因为不同数量级的浮点数, 在相加过程中, 需进行"对阶"操作. 
>> 将较小数目的小数点左移, 使得指数与最大的数目一致. \
>> 将两个以科学计数法表示的数, 徒手相加的过程中, 同样存在与这一"对阶"操作相同的环节. 
>
> 若不同数目间数量级差异悬殊, 数量级较小的数目, 将在对阶后丢失更多的有效数字. 

In [96]:
print(yFlt == yFlt2Dec, yFlt2Dec == yStr2Dec)
print([y * 39 / 13 for y in [yFlt, yFlt2Dec, yStr2Dec]])

True False
[0.6000000000000001, Decimal('0.6000000000000000333066907388'), Decimal('0.6')]


### `Decimal`的精度与修约管理
以小数的字符串形式构造的`Decimal`对象, 还将指定十进制小数的有效数字的数目. 
> 不同有效数字的`Decimal`对象, 在部分初等运算(包括四则运算, `Decimal`对象的`exp`, `ln`, `log10`等方法)所得的结果, 将采取"四舍六入五留双"的修约措施, 保留的有效数字也将相应地存在差异. 

由于以上特性, `Decimal`也可被应用于部分科学计算领域. 用于在部分科研实验数据的处理和分析过程中, 解决与有效数字, 以及特殊修约方法有关的要求. 

In [94]:
[(Decimal(deci_descr) ** 2, Decimal(deci_descr).sqrt()) \
    for deci_descr in ["2.56" + "0" * digit for digit in range(0, 10)]]

[(Decimal('6.5536'), Decimal('1.6')),
 (Decimal('6.553600'), Decimal('1.60')),
 (Decimal('6.55360000'), Decimal('1.60')),
 (Decimal('6.5536000000'), Decimal('1.600')),
 (Decimal('6.553600000000'), Decimal('1.600')),
 (Decimal('6.55360000000000'), Decimal('1.6000')),
 (Decimal('6.5536000000000000'), Decimal('1.6000')),
 (Decimal('6.553600000000000000'), Decimal('1.60000')),
 (Decimal('6.55360000000000000000'), Decimal('1.60000')),
 (Decimal('6.5536000000000000000000'), Decimal('1.600000'))]